# Synthetic Data Exploration

This notebook demonstrates generating and exploring synthetic field data for testing the Mneme pipeline.

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Mneme imports
import sys
sys.path.append('../src')

from mneme.data.generators import SyntheticFieldGenerator
from mneme.analysis.visualization import FieldVisualizer

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 8)

## 1. Generate Static Fields

In [ ]:
# Initialize generator
generator = SyntheticFieldGenerator(field_type='gaussian_random', seed=42)

# Generate different types of fields
field_types = {
    'gaussian_blob': {'n_centers': 3, 'sigma': 20},
    'sinusoidal': {'frequency': 0.1, 'angle': 45},
    'turbulent': {'scale': 10, 'intensity': 1.0},
    'reaction_diffusion': {'a': 0.16, 'b': 0.08}
}

fields = {}
for field_type, params in field_types.items():
    generator.field_type = field_type
    fields[field_type] = generator.generate_static(
        shape=(128, 128),
        parameters=params
    )

In [ ]:
# Visualize generated fields
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

for idx, (name, field) in enumerate(fields.items()):
    im = axes[idx].imshow(field, cmap='viridis', aspect='auto')
    axes[idx].set_title(f'{name.replace("_", " ").title()} Field')
    plt.colorbar(im, ax=axes[idx])

plt.tight_layout()
plt.show()

## 2. Generate Dynamic Fields

In [ ]:
# Generate time-evolving field
dynamic_params = {
    'drift_velocity': (0.5, 0.3),
    'growth_rate': 0.02,
    'noise_level': 0.1
}

dynamic_field = generator.generate_dynamic(
    shape=(64, 64),
    timesteps=50,
    parameters=dynamic_params
)

print(f"Dynamic field shape: {dynamic_field.shape}")

In [ ]:
# Visualize snapshots
time_indices = [0, 10, 25, 49]
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for idx, t in enumerate(time_indices):
    im = axes[idx].imshow(dynamic_field[t], cmap='viridis', aspect='auto')
    axes[idx].set_title(f'Time = {t}')
    plt.colorbar(im, ax=axes[idx])

plt.tight_layout()
plt.show()

## 3. Add Realistic Noise

In [ ]:
# Add different types of noise
clean_field = fields['gaussian_blob'].copy()

noise_types = {
    'gaussian': generator.add_noise(clean_field, noise_level=0.1),
    'salt_pepper': generator.add_noise(clean_field, noise_level=0.05, noise_type='salt_pepper'),
    'poisson': generator.add_noise(clean_field, noise_level=0.1, noise_type='poisson')
}

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.ravel()

# Original
im = axes[0].imshow(clean_field, cmap='viridis', aspect='auto')
axes[0].set_title('Original Field')
plt.colorbar(im, ax=axes[0])

# Noisy versions
for idx, (name, noisy_field) in enumerate(noise_types.items(), 1):
    im = axes[idx].imshow(noisy_field, cmap='viridis', aspect='auto')
    axes[idx].set_title(f'{name.replace("_", " ").title()} Noise')
    plt.colorbar(im, ax=axes[idx])

plt.tight_layout()
plt.show()

## 4. Generate Bioelectric-like Patterns

In [ ]:
# Generate planarian-inspired bioelectric pattern
bioelectric_params = {
    'anterior_voltage': -50,  # mV
    'posterior_voltage': -20,  # mV
    'gradient_steepness': 0.1,
    'lateral_variation': 10,
    'noise_level': 5
}

# Create anterior-posterior gradient
shape = (256, 128)
y_coords = np.linspace(0, 1, shape[0])
x_coords = np.linspace(0, 1, shape[1])
X, Y = np.meshgrid(x_coords, y_coords)

# Base gradient
gradient = bioelectric_params['anterior_voltage'] + \
           (bioelectric_params['posterior_voltage'] - bioelectric_params['anterior_voltage']) * Y

# Add lateral variation
lateral = bioelectric_params['lateral_variation'] * np.sin(2 * np.pi * X)

# Combine and add noise
bioelectric_field = gradient + lateral
bioelectric_field = generator.add_noise(bioelectric_field, bioelectric_params['noise_level'])

# Visualize
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(bioelectric_field, cmap='RdBu_r', aspect='auto')
ax.set_title('Synthetic Bioelectric Field (Planarian-like)')
ax.set_xlabel('Lateral axis')
ax.set_ylabel('Anterior-Posterior axis')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Voltage (mV)')
plt.show()

## 5. Save Generated Data

In [ ]:
# Save synthetic data for later use
output_dir = Path('../data/synthetic')
output_dir.mkdir(parents=True, exist_ok=True)

# Save as numpy arrays
np.savez(
    output_dir / 'synthetic_fields.npz',
    static_fields=fields,
    dynamic_field=dynamic_field,
    bioelectric_field=bioelectric_field,
    metadata={
        'field_types': field_types,
        'dynamic_params': dynamic_params,
        'bioelectric_params': bioelectric_params
    }
)

print(f"Saved synthetic data to {output_dir / 'synthetic_fields.npz'}")

## Next Steps

1. Use these synthetic fields to test reconstruction methods
2. Apply topology analysis to identify features
3. Test attractor detection on dynamic fields
4. Validate pipeline components before applying to real data